In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import sys
import torch
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, ChebConv
import matplotlib.pyplot as plt

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
dataset_path = 'simulator_data_1.csv'
# dataset_path = '/content/simulator_data.csv'
# df = pd.read_csv(dataset_path)


In [3]:
# Load the dataset
# dataset_path = "path_to_your_dataset.csv"  # Replace with your actual dataset path
data = pd.read_csv(dataset_path)

# Function to compute the distance between two points
def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


In [4]:
# Function to check for handovers between timestamps
def check_handovers(df, current_timestamp, next_timestamp):
    current_data = df[df['timestamp'] == current_timestamp]
    next_data = df[df['timestamp'] == next_timestamp]

    handovers = {}
    for vid in current_data['vehicleId'].unique():
        current_master = current_data[current_data['vehicleId'] == vid]['masterId'].iloc[0]
        next_master = next_data[next_data['vehicleId'] == vid]['masterId'].iloc[0] if vid in next_data['vehicleId'].values else current_master
        handovers[vid] = current_master != next_master

    return handovers

In [5]:
# Function to create a graph data object and NetworkX graph for a given timestamp
def create_graph_data_for_timestamp(df, timestamp):
    timestamp_data = df[df['timestamp'] == timestamp]
    vehicle_ids = timestamp_data['vehicleId'].unique()
    print('number of vehicles:', vehicle_ids.size)
    tower_ids = timestamp_data['towerId'].unique()
    print('number of towers:', tower_ids.size)
    vehicle_mapping = {vid: i for i, vid in enumerate(vehicle_ids)}
    tower_mapping = {tid: i + len(vehicle_ids) for i, tid in enumerate(tower_ids)}

    node_features = []
    edge_index = []
    edge_features = []

    G = nx.Graph()  # NetworkX Graph for visualization

    # Process vehicle nodes
    for vid in vehicle_ids:
        try:
            vehicle_data = timestamp_data[timestamp_data['vehicleId'] == vid].iloc[2]
        except IndexError:
            try:
                vehicle_data = timestamp_data[timestamp_data['vehicleId'] == vid].iloc[1]
            except IndexError:
                vehicle_data = timestamp_data[timestamp_data['vehicleId'] == vid].iloc[0]
        vehicle_data = timestamp_data[timestamp_data['vehicleId'] == vid].iloc[0]
        vehicle_features = [
            vehicle_data['vehicleSpeed'], vehicle_data['vehicleDirection'], vehicle_data['vehiclePosX'],
            vehicle_data['vehiclePosY']
        ]
        node_features.append(vehicle_features)
        G.add_node(vehicle_mapping[vid], speed=vehicle_data['vehicleSpeed'], dir=vehicle_data['vehicleDirection'], 
                  pos=(vehicle_data['vehiclePosX'], vehicle_data['vehiclePosY']),
                   type='vehicle', label=f'{vid}')

    # Process tower nodes
    for tid in tower_ids:
        master_data = timestamp_data[timestamp_data['masterId'] == tid][::-1]
        if not master_data.empty:
            tower_data = master_data.iloc[0]
            tower_features = [
                tower_data['masterPosX'], tower_data['masterPosY'],
                tower_data['masterLoad'], 0.0  # Placeholder values for missing features
            ]
            G.add_node(tower_mapping[tid], pos=(tower_data['masterPosX'], tower_data['masterPosY']), load=tower_data['masterLoad'],
                    type='tower', label=f'T{tid}')
        else:
            tower_data = timestamp_data[timestamp_data['towerId'] == tid][::-1].iloc[0]
            tower_features = [
                tower_data['towerPosX'], tower_data['towerPosY'],
                tower_data['towerLoad'], 0.0  # Placeholder values for missing features
            ]
            G.add_node(tower_mapping[tid], pos=(tower_data['towerPosX'], tower_data['towerPosY']), load=tower_data['towerLoad'],
                        type='tower', label=f'T{tid}')

        node_features.append(tower_features)
        

    # Add edges based on the masterId
    for _, row in timestamp_data.iterrows():
        vehicle_id = vehicle_mapping[row['vehicleId']]
        tower_id = tower_mapping[row['masterId']]  # Connect to the master tower
        edge_index.append([vehicle_id, tower_id])
        

        distance = calculate_distance(row['vehiclePosX'], row['vehiclePosY'],
                                      row['masterPosX'], row['masterPosY'])
        edge_features.append([row['throughput'], row['masterRssi'], distance])

        # Add edges to the NetworkX graph
        G.add_edge(vehicle_id, tower_id, throughput=row['throughput'],  rssi=row['masterRssi'], distance=distance)
    
    node_features_tensor = torch.tensor(node_features, dtype=torch.float)
    edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    # print(edge_index_tensor)
    # print(len(edge_index))
    edge_features_tensor = torch.tensor(edge_features, dtype=torch.float)

    graph_data = Data(x=node_features_tensor, edge_index=edge_index_tensor, edge_attr=edge_features_tensor)
    print("node features: ", graph_data.x.shape,"edge index: ", graph_data.edge_index.shape,"edge attr: ", graph_data.edge_attr.shape)
    return graph_data, G, vehicle_mapping, tower_mapping

In [6]:
def generate_sequences(df, sequence_length):
    """
    Generate sequences of graph data objects for the given sequence length.
    Each sequence consists of consecutive timestamps data.
    """
    timestamps = sorted(df['timestamp'].unique())
    print(len(timestamps))
    print(timestamps)
    
    sequences = []
# 202 - 20 = 182  0+0 =0   0+1 = 1  ... 0 + 20 = 20 / 182*20
# 21 - 20 = 1

#0+0 / 0+1 / 0+2 / ... / 0+19 [0,19]
#1+0 / 1+1 / 1+2 / ... / 1+19 [1,20]
    for i in range(len(timestamps) - sequence_length):
        print(i)
        sequence_graphs = []
        for j in range(sequence_length):
            print(j)
            timestamp = timestamps[i + j]
            graph_data, _, _, _ = create_graph_data_for_timestamp(df, timestamp)
            sequence_graphs.append(graph_data)
        sequences.append(sequence_graphs)

    return sequences

# Generate sequences
sequences = generate_sequences(data,20)


21
[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5]
0
0
number of vehicles: 31
number of towers: 10
node features:  torch.Size([41, 4]) edge index:  torch.Size([2, 54]) edge attr:  torch.Size([54, 3])
1
number of vehicles: 81
number of towers: 10
node features:  torch.Size([91, 4]) edge index:  torch.Size([2, 146]) edge attr:  torch.Size([146, 3])
2
number of vehicles: 99
number of towers: 10
node features:  torch.Size([109, 4]) edge index:  torch.Size([2, 182]) edge attr:  torch.Size([182, 3])
3
number of vehicles: 99
number of towers: 10
node features:  torch.Size([109, 4]) edge index:  torch.Size([2, 182]) edge attr:  torch.Size([182, 3])
4
number of vehicles: 99
number of towers: 10
node features:  torch.Size([109, 4]) edge index:  torch.Size([2, 182]) edge attr:  torch.Size([182, 3])
5
number of vehicles: 99
number of towers: 10
node features:  torch.Size([109, 4]) edge index:  torch.Size([2, 182]) edge attr:  torch.Size([

In [146]:
print(sequences)

[[Data(x=[41, 4], edge_index=[2, 54], edge_attr=[54, 3]), Data(x=[91, 4], edge_index=[2, 146], edge_attr=[146, 3]), Data(x=[109, 4], edge_index=[2, 182], edge_attr=[182, 3]), Data(x=[109, 4], edge_index=[2, 182], edge_attr=[182, 3]), Data(x=[109, 4], edge_index=[2, 182], edge_attr=[182, 3]), Data(x=[109, 4], edge_index=[2, 182], edge_attr=[182, 3]), Data(x=[109, 4], edge_index=[2, 182], edge_attr=[182, 3]), Data(x=[109, 4], edge_index=[2, 182], edge_attr=[182, 3]), Data(x=[110, 4], edge_index=[2, 184], edge_attr=[184, 3]), Data(x=[110, 4], edge_index=[2, 185], edge_attr=[185, 3]), Data(x=[110, 4], edge_index=[2, 186], edge_attr=[186, 3]), Data(x=[110, 4], edge_index=[2, 185], edge_attr=[185, 3]), Data(x=[110, 4], edge_index=[2, 183], edge_attr=[183, 3]), Data(x=[110, 4], edge_index=[2, 183], edge_attr=[183, 3]), Data(x=[110, 4], edge_index=[2, 183], edge_attr=[183, 3]), Data(x=[110, 4], edge_index=[2, 183], edge_attr=[183, 3]), Data(x=[110, 4], edge_index=[2, 184], edge_attr=[184, 3]),

In [147]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class TGCNUnsupervised(nn.Module):
    def __init__(self, node_feature_size, edge_feature_size, hidden_size):
        super(TGCNUnsupervised, self).__init__()
        self.conv1 = GCNConv(node_feature_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
        self.gru = nn.GRU(hidden_size , hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, edge_feature_size)  # Predicting future edge features

    def forward(self, data_list):
        x_all, edge_index_all, edge_attr_all = [], [], []
        max_num_nodes = max(data.x.size(0) for data in data_list)
        for data in data_list:
            x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
            num_nodes = x.size(0)
            padding_size = max_num_nodes - num_nodes
            padding = torch.zeros(padding_size, x.size(1)).to(x.device)
            x = torch.cat([x, padding], dim=0)
            x = F.relu(self.conv1(x, edge_index))
            x = F.relu(self.conv2(x, edge_index))
            x_all.append(x)
            edge_index_all.append(edge_index)
            edge_attr_all.append(edge_attr)
        x_all = torch.stack(x_all)
        out, _ = self.gru(x_all)
        pred_edge_attr = self.fc(out)
        return pred_edge_attr


In [148]:
model = TGCNUnsupervised(4, 3, hidden_size=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [149]:
def train_unsupervised(model, data_sequences, optimizer):
    model.train()
    for sequence in data_sequences:  # Each contains graphs for 20 timestamps + 1 for target
        optimizer.zero_grad()

        # Assuming your model outputs predictions for the sequence
        pred_edge_attr_seq = model(sequence[:-1])  # Model processes all but the last graph for prediction

        # The target sequence's edge attributes
        actual_edge_attr = sequence[-1].edge_attr

        # Assuming pred_edge_attr_seq[-1] and actual_edge_attr are tensors
        min_edges = min(pred_edge_attr_seq[-1].size(0), actual_edge_attr.size(0))

        # Compute loss only over the common set of edges
        loss = F.mse_loss(pred_edge_attr_seq[-1][:min_edges], actual_edge_attr[:min_edges])

        loss.backward()
        optimizer.step()

    print("Training Completed")


In [150]:
# train_unsupervised(model, sequences, optimizer)
train_unsupervised(model, sequences, optimizer)


Training Completed


In [ ]:
def infer_handover_targets(model, current_sequence):
    """
    Infer handover targets for each vehicle in the sequence.

    :param model: Trained TGCN model.
    :param current_sequence: Sequence of graph data leading up to the current timestamp.
    :return: Dict mapping vehicle IDs to their recommended handover tower IDs.
    """
    pred_edge_attr = model(current_sequence)[-1]  # Get predictions for the next timestamp
    handover_decisions = {}

    for i, edge in enumerate(current_sequence[-1].edge_index.t()):
        vehicle_id, tower_id = edge.tolist()
        # Assuming edge attributes include predicted signal quality and distance
        predicted_signal_quality, predicted_distance = pred_edge_attr[i]

        # Calculate score (example calculation, adjust according to your needs)
        score = predicted_signal_quality - predicted_distance

        if vehicle_id not in handover_decisions or handover_decisions[vehicle_id]['score'] < score:
            handover_decisions[vehicle_id] = {'tower_id': tower_id, 'score': score}

    # Select the tower with the highest score for each vehicle
    recommended_handovers = {v: d['tower_id'] for v, d in handover_decisions.items()}
    return recommended_handovers